In [ ]:

import requests
import json

import os
from openai import OpenAI
from dotenv import load_dotenv

client = OpenAI()

# Set your OpenAI API key here
token = os.getenv("SECRET")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

# Define the function that will be called
def get_birzai_info():
    response = requests.get(
        "https://en.wikipedia.org/w/api.php",
        params={
            "action": "query",
            "format": "json",
            "prop": "extracts",
            "exintro": True,
            "explaintext": True,
            "titles": "Biržai"
        }
    )
    data = response.json()
    page = next(iter(data["query"]["pages"].values()))
    return page["extract"]

# Define the function schema
functions = [
    {
        "name": "get_birzai_info",
        "description": "Get a short summary about Biržai from Wikipedia.",
        "parameters": {
            "type": "object",
            "properties": {},
        },
    }
]

# Send a message to the model
message = [{"role": "user", "content": "Tell me about Biržai"}]

response = openai.ChatCompletion.create(
    model=model
    messages=message,
    functions=functions,
    function_call="auto"
)

response_message = response["choices"][0]["message"]

# Check if the model decided to call the function
if response_message.get("function_call"):
    function_name = response_message["function_call"]["name"]

    if function_name == "get_birzai_info":
        # Call the function
        function_response = get_birzai_info()

        # Append function result back to messages
        message.append(response_message)
        message.append({
            "role": "function",
            "name": function_name,
            "content": function_response,
        })

        # Ask model to continue with function results
        final_response = openai.ChatCompletion.create(
            model=model,
            messages=message
        )

        print(final_response["choices"][0]["message"]["content"])
    else:
        print("Function not recognized.")
else:
    print(response_message["content"])
